### 필요한 라이브러리 불러오기 및 상수 선언

In [47]:
import numpy as np
import pandas as pd
import urllib.request
from PIL import Image
import os

import warnings
warnings.filterwarnings('ignore')

### 데이터셋 가져오기

In [3]:
YOUTUBE_LIST = ['과학드림', '곽튜브', '리뷰엉이', '배말랭', '백종원', '보다', '비글부부', '서기채널', '셜록현준', '슈카월드', '알간지', '영국남자', '옐언니', '오킹TV', '오사카에사는사람들TV', '웅이', '육식맨', '주연', '지식해적단', '쯔양', '청담언니', '침착맨', '테스터훈', '트래블튜브', '피지컬갤러리', '햄지', '허팝', '혜진쓰', 'CreativeDen', 'ITSub잇섭', 'LeoJ Makeup', 'SOD', 'UNDERkg']

In [4]:
# path 설정
DATA_PATH = 'C:/Users/sungh/.vscode/repository/thumbnailv2/Thumbnails-Project/datasets/thumbnail_csv/'

### 함수 선언

In [66]:
# 1사분위수와 비교했을 때, 더 작으면 유의미하게 안 좋은 썸네일,
# 3사분위수와 비교했을 때, 더 크면 유의미하게 좋은 썸네일으로 라벨링하는 함수
# 나머지는 삭제한다

def getLabels13Quartiles(df):
    df['label'] = 0
    Q1View = np.percentile(df['ViewCount'], 25)
    Q3View = np.percentile(df['ViewCount'], 75)
    
    for i in range(len(df)):
        # if ViewCount is higher than Q3 then it is labeled as 1
        if df['ViewCount'][i] >= Q3View:
            df['label'][i] = 1
        
        # else if ViewCount is lower than Q1 then it is labeled as 0
        elif df['ViewCount'][i] < Q1View:
            df['label'][i] = 0
            
        # else ViewCount is between Q1 and Q3 so it is dropped
        else:
            df['label'][i] = pd.NA
    df.dropna(inplace=True)
    return df

In [16]:
#get image from url 
def url_to_img(dataset, IDX):
    for i in range(len(dataset)):
        img_id = dataset['Unnamed: 0'][i]
        img_url = dataset['HDThumbnailUrl'][i]

        if dataset['label'][i]==0:
            file_name = IMG_PATH + f"/0/{IDX+1}_{img_id}.png"
        else:
            file_name = IMG_PATH + f"/1/{IDX+1}_{img_id}.png"

        urllib.request.urlretrieve(img_url, file_name)
        print(f"{file_name}.png saved")   

### 라벨링 및 이미지 저장

In [17]:
fullData = pd.DataFrame({
    'ChannelName' : [],
    'VideoIndex': [],
    'VideoID' : [],
    'VideoTitle': [],
    'Label': [],
})

datas = []
zeros = []
ones = []

In [67]:
for idx in range(len(YOUTUBE_LIST)):
    NAME = YOUTUBE_LIST[0]

    data = pd.read_csv(DATA_PATH + NAME + '.csv')

    # index 초기화 및 불필요한 column 제거
    data['Published'] = pd.to_datetime(data.Published)
    data = data.sort_values(by='Published')
    data.reset_index(inplace=True)
    data.drop(['index'], axis=1, inplace=True)

    dataLabeled = getLabels13Quartiles(data)

    # 영상 정보 저장
    datas.append( len(dataLabeled) )
    zeros.append( len(dataLabeled[dataLabeled['label']==0]) )
    ones.append( len(dataLabeled[dataLabeled['label']==1]) )

    mainData = pd.DataFrame({
        'ChannelName' : [NAME for i in range(len(dataLabeled))],
        'VideoIndex': dataLabeled['Unnamed: 0'],
        'VideoID' : dataLabeled['VideoId'],
        'VideoTitle': dataLabeled['Title'],
        'Label':dataLabeled['label']
    })
    fullData = pd.concat([fullData, mainData])

    # 이미지 저장
    IMG_PATH = 'D:/_YBIGTA/labeled_thumbnails/' + NAME

    os.makedirs(IMG_PATH, exist_ok=True)
    os.makedirs(IMG_PATH + '/0', exist_ok=True)
    os.makedirs(IMG_PATH + '/1', exist_ok=True)

    url_to_img(dataLabeled, idx)

(70, 7)
    Unnamed: 0                 Published  \
4          134 2019-04-19 08:57:53+00:00   
6          132 2019-05-03 09:09:52+00:00   
7          131 2019-05-10 09:00:06+00:00   
8          130 2019-05-17 09:00:09+00:00   
11         127 2019-06-07 09:00:00+00:00   

                                      HDThumbnailUrl  ViewCount      VideoId  \
4   https://i.ytimg.com/vi/oAlmpXS9wPw/hqdefault.jpg    3427519  oAlmpXS9wPw   
6   https://i.ytimg.com/vi/hswtPZSnKo8/hqdefault.jpg     104774  hswtPZSnKo8   
7   https://i.ytimg.com/vi/kbiOUSGeeME/hqdefault.jpg     210557  kbiOUSGeeME   
8   https://i.ytimg.com/vi/8pd4W9AqrbE/hqdefault.jpg     168150  8pd4W9AqrbE   
11  https://i.ytimg.com/vi/M2rjIgmj87g/hqdefault.jpg     346478  M2rjIgmj87g   

                                           Title label  
4           기생충은 왜 개구리 다리를 이렇게 만들었을까?(기생충의 과학-1)     1  
6                사랑의 유통기한, 있을까? 없을까? (feat. 호르몬)     0  
7                              아기는 왜 귀여울까? (^.^)     0  
8                 

### 영상 정보 저장

len(fullData)

In [58]:
info = pd.DataFrame({
    'youtube' : YOUTUBE_LIST[0],
    'num_of_data' : datas[0],
    'label 0' : zeros,
    'label 1' : ones
})

In [59]:
info

,youtube,num_of_data,label 0,label 1
0,과학드림,139,69,70
1,과학드림,139,69,70
2,과학드림,139,35,104
3,과학드림,139,35,35


In [84]:
info.to_csv('C:/Users/sungh/.vscode/repository/thumbnailv2/Thumbnails-Project/sunghj1118/upgrades/data_information.csv')
fullData.to_csv('C:/Users/sungh/.vscode/repository/thumbnailv2/Thumbnails-Project/sunghj1118/upgrades/full_data_label.csv')